## 爬虫豆瓣图书TOP250作业

#### 爬取数据并且保存

In [1]:
import requests
import bs4
import pandas as pd

In [2]:
def get_name(name_list):
    name = ''
    for item in name_list:
        name = name + str(item)
    if name.find('[') != -1 and name.find(']') != -1:
        country = name[name.index('[') + 1:name.index(']')]
        name = name[name.index(']') + 1:].replace("著", "").replace(" ", "")
    else:
        country = '中'
    return name, country

In [3]:
def search(req, i, result):
    soup = bs4.BeautifulSoup(req, "html.parser").find_all(name="tr", attrs={'class':'item'})
    for item, soup_item in enumerate(soup):
        # print(str(soup_item.find(name="div", attrs={'class':'pl2'}).select("body a"))\
        #     .replace(" ", '').replace("\n", ''))
        result.loc[i*25 + item, '书名'] = str(str(soup_item.find(name="div", attrs={'class':'pl2'})
                                                .select("body a")[0]['title'])).replace(" ", '').replace("\n", '')
        result.loc[i*25 + item, '链接'] = str(soup_item.select("body a")[1]["href"])
        result.loc[i*25 + item, '是否可试读'] = \
            str(soup_item.select("body img")[1]['title']) if len(soup_item.select("body img")) >= 2 else "不可试读"
        result.loc[i*25 + item, '价格'] = \
            str(soup_item.find(name="p", attrs={'class':'pl'}).string).split("/")[-1].replace("元", '')
        result.loc[i*25 + item, '出版时间'] = \
            str(soup_item.find(name="p", attrs={'class':'pl'}).string).split("/")[-2]
        result.loc[i*25 + item, '出版社'] = \
            str(soup_item.find(name="p", attrs={'class':'pl'}).string).split("/")[-3]
        name, country = get_name(str(soup_item.find(name="p", attrs={'class':'pl'}).string).split("/")[:-3])
        result.loc[i*25 + item, '作者'] = name
        result.loc[i*25 + item, '作者国籍'] = country
        result.loc[i*25 + item, '评价'] = str(soup_item.find(name="span", attrs={'class':'inq'}).string) if \
            soup_item.find(name="span", attrs={'class':'inq'}) is not None else '无热评'
        result.loc[i*25 + item, '评价人数'] = \
            str(soup_item.find(name="span", attrs={'class':'pl'}).string)\
                .replace(" ", '').replace("\n", '').replace("人评价)", '').replace("(", '')
        result.loc[i*25 + item, '别名'] = \
            '无别名' if soup_item.find(name="span", attrs={'style': "font-size:12px;"}) is None \
                else str(soup_item.find(name="span", attrs={'style': "font-size:12px;"})
                    .string).replace(":", '')
    return result

In [4]:
# 根据url获取数据
def req(url, i, result):
    header = {
        'User-Agent': r'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
    }
    req = requests.get(url, headers=header, timeout=5).text.encode("utf-8")
    result = search(req, i, result)
    return result
    # try:
    #     req = requests.get(url, headers=header, timeout=5).text.encode("utf-8")
    #     search(req, i, result)
    # except Exception as e:
    #     print("except", e)

In [5]:
def get_req():
    baseurl = "https://book.douban.com/top250?"
    result = pd.DataFrame(columns=['书名', '别名', '作者', '作者国籍', '出版社',
                                   '出版时间', '价格', '评价', '评价人数', '是否可试读', '链接'])
    # 获取数据
    for i in range(10):
        url = baseurl + 'start=' + str(i*25)
        result = req(url, i, result)
    return result

In [6]:
result = get_req()
result

,书名,别名,作者,作者国籍,出版社,出版时间,价格,评价,评价人数,是否可试读,链接
0,红楼梦,无别名,曹雪芹,清,人民文学出版社,1996-12,59.70,都云作者痴，谁解其中味？,366932,可试读,https://book.douban.com/subject/1007305/
1,活着,无别名,余华,中,作家出版社,2012-8-1,20.00,生的苦难与伟大,682644,可试读,https://book.douban.com/subject/4913064/
2,1984,Nineteen Eighty-Four,乔治·奥威尔刘绍铭,英,北京十月文艺出版社,2010-4-1,28.00,栗树荫下，我出卖你，你出卖我,235328,不可试读,https://book.douban.com/subject/4820710/
3,百年孤独,Cien años de soledad,加西亚·马尔克斯范晔,哥伦比亚,南海出版公司,2011-6,39.50,魔幻现实主义文学代表作,371052,不可试读,https://book.douban.com/subject/6082808/
4,三体全集,地球往事三部曲,刘慈欣,中,重庆出版社,2012-1-1,168.00,地球往事三部曲,130113,不可试读,https://book.douban.com/subject/6518605/
...,...,...,...,...,...,...,...,...,...,...,...
245,显微镜下的大明,无别名,马伯庸,中,湖南文艺出版社,2019-1,52.00,无热评,24287,不可试读,https://book.douban.com/subject/30414743/
246,给青年诗人的信,雅众·文学馆,里尔克冯至雅众文化,奥,云南人民出版社,2015-12,29.00,无热评,10870,可试读,https://book.douban.com/subject/26685994/
247,佐贺的超级阿嬷,无别名,岛田洋七陈宝莲,日,南海出版公司,2007-3,20.00,乐观的外婆却总有神奇的办法,22844,可试读,https://book.douban.com/subject/2008284/
248,人类的群星闪耀时,Sternstunden der Menschheit Vierzehn historisc...,斯蒂芬·茨威格舒昌善,奥,广西师范大学出版社,2004-8,18.00,十四个影响人类文明的瞬间,28141,不可试读,https://book.douban.com/subject/1083762/


In [7]:
result.to_csv('./data.csv', index=False)

#### 分析

In [2]:
# 导入库
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [3]:
data_csv = pd.read_csv('./data.csv')
data_csv

,书名,别名,作者,作者国籍,出版社,出版时间,价格,评价,评价人数,是否可试读,链接
0,红楼梦,无别名,曹雪芹,清,人民文学出版社,1996-12,59.70,都云作者痴，谁解其中味？,366932,可试读,https://book.douban.com/subject/1007305/
1,活着,无别名,余华,中,作家出版社,2012-8-1,20.00,生的苦难与伟大,682644,可试读,https://book.douban.com/subject/4913064/
2,1984,Nineteen Eighty-Four,乔治·奥威尔刘绍铭,英,北京十月文艺出版社,2010-4-1,28.00,栗树荫下，我出卖你，你出卖我,235328,不可试读,https://book.douban.com/subject/4820710/
3,百年孤独,Cien años de soledad,加西亚·马尔克斯范晔,哥伦比亚,南海出版公司,2011-6,39.50,魔幻现实主义文学代表作,371052,不可试读,https://book.douban.com/subject/6082808/
4,三体全集,地球往事三部曲,刘慈欣,中,重庆出版社,2012-1-1,168.00,地球往事三部曲,130113,不可试读,https://book.douban.com/subject/6518605/
...,...,...,...,...,...,...,...,...,...,...,...
245,显微镜下的大明,无别名,马伯庸,中,湖南文艺出版社,2019-1,52.00,无热评,24287,不可试读,https://book.douban.com/subject/30414743/
246,给青年诗人的信,雅众·文学馆,里尔克冯至雅众文化,奥,云南人民出版社,2015-12,29.00,无热评,10870,可试读,https://book.douban.com/subject/26685994/
247,佐贺的超级阿嬷,无别名,岛田洋七陈宝莲,日,南海出版公司,2007-3,20.00,乐观的外婆却总有神奇的办法,22844,可试读,https://book.douban.com/subject/2008284/
248,人类的群星闪耀时,Sternstunden der Menschheit Vierzehn historisc...,斯蒂芬·茨威格舒昌善,奥,广西师范大学出版社,2004-8,18.00,十四个影响人类文明的瞬间,28141,不可试读,https://book.douban.com/subject/1083762/


In [25]:
data_csv_country = data_csv["作者国籍"].value_counts()
fig = {
  "data": [
    {
      "values": data_csv_country.values,
      "labels": data_csv_country.index,
      "domain": {"x": [0, .5]},
      "name": "Number Of",
      "hoverinfo":"label+percent",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"作者国家分布",
                'height':600,
                "width":800
    }
}
iplot(fig)

In [29]:
data_csv_press = data_csv["出版社"].value_counts(ascending=True)[-30:]
data_csv_press.loc['其它'] = sum(data_csv["出版社"].value_counts(ascending=True)[:-30].values)
trace1 = go.Bar(
 x = data_csv_press.index,
 y = data_csv_press,
 name = "citations",
 marker = dict(color = 'rgba(255, 174, 255, 0.5)',
 line=dict(color='rgb(0,0,0)',width=1.5)))

data = [trace1]

layout = go.Layout(barmode = "group", yaxis=dict(title= '图书数量',ticklen= 5,zeroline= False),
                   title="出版社分布")
fig = go.Figure(data = data, layout = layout)
iplot(fig)

In [49]:
data_csv_isRead = data_csv["是否可试读"].value_counts()

size = data_csv_isRead

data = [
 {
 'x':data_csv_isRead.index,
 'y':data_csv_isRead,
 'mode':'markers',
  'marker':{
 'color':[float(each) for each in data_csv_isRead],
 'size':data_csv_isRead,
 'showscale':True
 }
 }
]
layout = go.Layout(yaxis=dict(title= '图书数量',ticklen= 5,zeroline= False),
                   xaxis=dict(),
                   title="是否可试读", height=500, width=600)
fig = go.Figure(data = data, layout = layout)

iplot(fig)

In [60]:
data_csv_readNum = data_csv["评价人数"]

trace1 =go.Scatter(
 x = data_csv.index,
 y = data_csv["评价人数"],
  mode = "markers",
 name = "2014",
 marker = dict(color = 'rgba(128, 128, 255, 0.8)'),
 text= data_csv['书名'])

#3.定义layout对象
layout = dict(xaxis= dict(title= '书序列号',ticklen= 5,zeroline= False),
 yaxis= dict(title= '评价人数',ticklen= 5,zeroline= False)
 )
#4.将graph部分和layout部分组合成figure对象，绘图
fig = dict(data = trace1, layout = layout)
iplot(fig)